<a href="https://colab.research.google.com/github/tntnu/QA/blob/main/%5BNLP%5D_%EB%8F%84%EB%B0%B0_%ED%95%98%EC%9E%90_%EC%A7%88%EC%9D%98_%EC%9D%91%EB%8B%B5_%EC%B2%98%EB%A6%AC_%ED%95%9C%EC%86%94%EB%8D%B0%EC%BD%94_%EC%8B%9C%EC%A6%8C2_%EC%83%9D%EC%84%B1_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded_files = files.upload()

Saving train.csv to train.csv


In [ ]:
from google.colab import files
uploaded_files = files.upload()

Saving test.csv to test.csv


### 생성 모델 경량화

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from langchain.vectorstores import FAISS
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
# 양자화에 필요한 패키지
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 46.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 11.8 MB/s eta 0:00:00


In [ ]:
import accelerate, peft, trl
print("accelerate:", accelerate.__version__)
print("peft:", peft.__version__)
print("trl:", trl.__version__)

import bitsandbytes as bnb
print("bitsandbytes:",bnb.__version__)

accelerate: 1.11.0.dev0
peft: 0.17.2.dev0
trl: 0.22.1
bitsandbytes: 0.47.0


### 모델 경량화

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

model_id = 'microsoft/Phi-4-mini-instruct'
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token = True)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config, device_map = 'auto')

KeyboardInterrupt: 

### huggingface microsoft/Phi-4-mini-instruct 경량화 모델 불러오기

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "wjdgnl/Phi-4-mini-instruct-4bit"

# 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"  # GPU 환경이면 자동으로 적절한 dtype 사용
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

...

chat_template.jinja:   0%|          | 0.00/423 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

In [ ]:
# 경량화 잘 되었는지 확인
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(200064, 3072, padding_idx=199999)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=5120, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, ou

In [ ]:
# 잘 실행되는지 확인
device = 'cuda:0'

def get_completion(query : str, model, tokenizer) :
  prompt_template = '''<|system|>Your are a helpful assistant.<|end|><|user|>{query}<|end|><|assistant|>'''
  prompt = prompt_template.format(query = query)

  encoded = tokenizer(prompt, return_tensors = 'pt', add_special_tokens = True)
  inputs = encoded.to(device)
  generated_ids = model.generate(inputs['input_ids'], max_new_tokens = 512, do_sample = False)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens = True)
  return decoded

# Fine-tuning 이전 (test 데이터 첫번째 질문 사용)
result = get_completion(query = '방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?', model = model, tokenizer = tokenizer)
print(result)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Your are a helpful assistant.방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?방청 페인트에는 여러 종류가 있으며, 선택은 프로젝트의 요구 사항과 개인의 선호에 따라 달라집니다. 일반적인 종류는 다음과 같습니다:

1. 유리 페인트: 유리 페인트는 유리 표면에 부착하여 유리처럼 보이는 효과를 제공하는 페인트입니다. 유리 페인트는 다양한 색상과 패턴으로 제공되며, 유리 표면에 부착하는 데 사용됩니다.

2. 유리 페인팅: 유리 페인팅은 유리 표면에 부착하여 유리처럼 보이는 효과를 제공하는 페인트입니다. 유리 페인팅은 유리 표면에 부착하는 데 사용되며, 유리 페인트와 유사하게 색상과 패턴으로 제공됩니다.

3. 유리 페인팅: 유리 페인팅은 유리 표면에 부착하여 유리처럼 보이는 효과를 제공하는 페인트입니다. 유리 페인팅은 유리 표면에 부착하는 데 사용되며, 유리 페인트와 유사하게 색상과 패턴으로 제공됩니다.

원목사이딩은 다양한 목재를 사용하여 다양한 패턴과 스타일을 만들 수 있는 방법입니다. 그러나 원목사이딩에는 몇 가지 단점이 있습니다:

1. 시간 소모: 원목사이딩은 시간이 많이 소모되며, 프로젝트를 완료하는 데 시간이 걸릴 수 있습니다.

2. 비용: 원목사이딩은 다양한 목재를 사용하여 다양한 패턴과 스타일을 만들 수 있는 방법입니다. 그러나 다양한 목재를 사용하면 비용이 많이 들 수 있습니다.

3. 기술 요구 사항: 원목사이딩은 다양한 목재를 사용하여 다양한 패턴과 스타일을 만들 수 있는 방법입니다. 그러나 다양한 목재를 사용하면 비용이 많이 들 수 있습니다.

4. 손상 가능성: 원목사이딩은 다양한 목재를 사용하여 다양한 패턴과 스타일을 만들 수 있는 방법입니다. 그러나 다양한 목재를 사용하면 손상 가능성이 높아질 수 있습니다.

결론적으로, 선택은 프로젝트의 요구 사항과 개인의 선호에 따라 달라집니다. 원목사이딩은 다양한 목재를 사용하여 다양한 패턴과 스타일을 만들

### 데이터 불러오기 및 dataset 변환

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...
...,...,...,...,...,...,...,...,...,...
639,TRAIN_639,벽장 부위 결로의 원인이 뭐야?,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,타 마감하자,벽장 부위 결로의 원인은 난방이 이웃한 방과동일한 조건이 되나 그 방에 비해 저온인...,"벽장 부위 결로의 원인은 주로 난방이 자리잡은 방이 내부 온도가 낮은 반면, 외부 ...",벽장 부위 결로가 발생하는 원인은 난방이 잘 이루어지지 않아 해당 공간이 저온인 반...,"벽장 부위 결로가 발생하는 원인은 난방이 인접한 방과 같은 조건을 갖추고 있지만, ...",벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
640,TRAIN_640,"AD, PD에 면한 벽체 결로의 원인이 뭐야?",벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,타 마감하자,"AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 주된 원인은 외기에 접하는 면 좌, 우측 벽체에 단...","AD, PD에 면한 벽체 결로의 원인으로는 외기에 접하는 면 좌, 우측 벽체는 일반...","AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
641,TRAIN_641,외벽 모서리 부위에 결로가 발생하는 원인이 뭐야?,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,타 마감하자,외벽모서리 부위에 결로가 발생하는 원인은 높은 온도차 때문입니다. 외벽 모서리의 경...,외벽 모서리 부위에 결로가 발생하는 원인은 중요한 역할을 합니다. 외부에 비치는 세...,외벽 모서리 부위에 결로가 발생하는 원인으로는 높은 온도차가 대표적입니다. 외벽 모...,외벽 모서리 부위에 결로가 발생하는 원인은 실내와 외부의 온도차 때문입니다. 외벽 ...,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
642,TRAIN_642,창호 결로의 대책은 뭐야?,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,타 마감하자,창호결로의 대책은 제품을 선정할 때 KS에 규정된 프레임을 선정하고 유리의 열관류율...,창호 결로의 대책은 KS에 규정된 프레임을 선정하고 유리의 열관류율 및 결로 발생 ...,"창호 결로를 예방하기 위한 대책은 KS에 규정된 프레임을 선택하고, 열관류율이 적은...","창호 결로의 대책은 여러 가지가 있습니다. 먼저, 창호를 선택할 때 KS에 규정된 ...","창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [ ]:
train.columns

Index(['id', '질문_1', '질문_2', 'category', '답변_1', '답변_2', '답변_3', '답변_4',
       '답변_5'],
      dtype='object')

- 처음부터 질문+답변 묶음으로 데이터셋을 구성하는 것이 가장 간단하고 효과적입니다.

- 질문만, 답변만 따로 만들 필요는 없고, RAG를 결합하지 않는 이상 단일 시퀀스로 충분합니다.

In [ ]:
from datasets import Dataset

docs = []
for _, row in train.iterrows() :
  question_texts = [row['질문_1'], row['질문_2']]
  answer_texts = [row[f'답변_{i}'] for i in range(1, 6)]

  for q in question_texts:
    for a in answer_texts:
      docs.append({
          'text' : f"<|system|>You are a helpful assistant.<|end|><|user|>{q}<|end|><|assistant|>{a}"

      })

dataset = Dataset.from_pandas(pd.DataFrame(docs))
dataset

Dataset({
    features: ['text'],
    num_rows: 6440
})

In [ ]:
dataset[0]

{'text': '<|system|>You are a helpful assistant.<|end|><|user|>면진장치가 뭐야?<|end|><|assistant|>면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다.'}

In [ ]:
dataset[5]

{'text': '<|system|>You are a helpful assistant.<|end|><|user|>면진장치에 사용되는 주요 기술은 무엇인가요?<|end|><|assistant|>면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다.'}

### 데이터 분할

In [ ]:
dataset = dataset.train_test_split(test_size = 0.2, seed = 2025)
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 5152
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1288
    })
})

In [ ]:
train = dataset['train']
test = dataset['test']
print(train)

Dataset({
    features: ['text'],
    num_rows: 5152
})


In [ ]:
train[0]

{'text': '<|system|>You are a helpful assistant.<|end|><|user|>폴리싱 타일을 사용하는 장점은 무엇인가요?<|end|><|assistant|>폴리싱 타일은 내구성이 뛰어나고 변형 및 오염, 얼룩에 강해서 오랫동안 사용할 수 있는 것이 장점입니다. 또한, 표면이 매끄러워 깨끗하고 유지보수가 쉽다는 점도 장점으로 꼽힙니다.'}

### 데이터 토큰화

In [ ]:
def tokenizer_function(example) :
  return tokenizer(example['text'], truncation = True) # truncation=True : 토크나이저가 모델이 허용하는 최대 시퀀스 길이보다 긴 입력이 들어왔을 때 잘라내도록(Truncate) 하는 옵션

train = train.map(tokenizer_function, batched = True) # batched=True → 데이터 전처리 단계에서 여러 샘플을 한 번에 처리, 성능에 영향 미치지 않음
test = test.map(tokenizer_function, batched = True)

Map:   0%|          | 0/5152 [00:00<?, ? examples/s]

Map:   0%|          | 0/1288 [00:00<?, ? examples/s]

### 파인튜닝 전 성능 출력

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# 1. 테스트 데이터 불러오기
test_df = pd.read_csv("test.csv")   # 반드시 'id', '질문' 컬럼 포함

# 2. 답변 생성 함수 (파인튜닝 전 모델 활용)
def get_completion(query: str, model, tokenizer):
    # 프롬프트에는 system 역할 포함
    prompt_template = "<|system|>You are a helpful assistant.<|end|><|user|>{query}<|end|><|assistant|>"
    prompt = prompt_template.format(query=query)

    encodeds = tokenizer(prompt, return_tensors='pt', add_special_tokens=True)
    model_inputs = encodeds.to("cuda:0")

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,   # 답변 길이 제한
            do_sample=False
        )

    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # system 메시지 제거
    decoded = decoded.replace("You are a helpful assistant.", "").strip()
    # 질문 반복 제거
    if query in decoded:
        decoded = decoded.split(query)[-1].strip()
    # 혹시 <|assistant|> 토큰이 남아있으면 제거
    if "<|assistant|>" in decoded:
        decoded = decoded.split("<|assistant|>")[-1].strip()

    return decoded

# 3. 임베딩 모델 로드 (512차원 출력)
embedding_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

# 4. 테스트 데이터 → 답변 생성 → 임베딩 변환
embeddings_list = []
answers_list = []

for q in tqdm(test_df["질문"], desc="질문 처리 중 (파인튜닝 전 모델)"):
    # ✅ trainer.model 대신 초기 로드한 model 사용
    answer = get_completion(q, model, tokenizer)
    vec = embedding_model.encode(answer)
    embeddings_list.append(vec)
    answers_list.append(answer)

    # 진행 상황 출력
    print("\n📌 질문:", q)
    print("💡 답변:", answer)
    print("-" * 80)

# 5. 임베딩을 DataFrame으로 변환
embedding_df = pd.DataFrame(embeddings_list, columns=[f"dim_{i}" for i in range(512)])

# 6. 제출용 DataFrame
submission = pd.concat([test_df[["id"]], embedding_df], axis=1)

# 7. CSV 저장
submission.to_csv("submission_baseline.csv", index=False)
print("✅ submission_baseline.csv 저장 완료! (파인튜닝 전 모델 결과)")


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...


📌 질문: 평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어떤 장점이 있나요?
💡 답변: 수성 벽체용 탄성 방수 도료를 사용하면 여러 가지 장점이 있습니다:

1. 보호: 수성 벽체용 탄성 방수 도료는 벽체에 물 방출을 방지하여 물 손실을 줄이고 수성 벽체의 수명을 연장할 수 있습니다.

2. 유지 관리: 수성 벽체용 탄성 방수 도료는 벽체 표면에 물 방출을 방지하여 유지 관리가 용이합니다. 수성 벽체 표면에 물 방출이 발생하면 표면이 부패하고 부패가 진행하면 수성 벽체의 수명이 줄어듭니다.

3. 비용 효율성: 수성 벽체용 탄성 방수 도료를 사용하면 수성 벽체의 수명을 연장할 수 있으며 장기적으로 비용 효율성이 높아집니다. 수성 벽체 표면에 물 방출이 발생하면 수성 벽체의 수명이 줄어듭니다. 수성 벽체 표면에 물 방출이 발생하면 수성 벽체의 수명이 줄어듭니다. 수성 벽체 표면에 물 방출이 발생하면 수성 벽체의 수명이 줄어듭니다. 수성 벽체 표면에 물 방출이 발생하면 수성 벽체의 수명이 줄어듭니다. 수성 벽체 표면에 물 방출이 발생하면 수성 벽체의 수명이 줄어듭니다. 수성 벽체 표면에 물 방출이 발생하면 수성 벽체의 수명이 줄어듭니다. 수성 벽체 표면에 물 방출이 발생하면 수성 벽체의 수명이 줄어듭니다. 수성 벽체 표면에 물 방출이 발생하면 수성 벽체의 수명이 줄어듭니다. 수성 벽체 표면에 물 방출이 발생하면 수성 벽체의 수명이 줄어듭니다. 수성 벽체 표면에 물 방출이 발생하면 수성 벽체의 수명이 줄어듭니다. 수성 벽체 표면에 물 방출이 발생하면 수성 벽체의 수명이 줄어듭니다. 수성 벽체 표면에 물 방
--------------------------------------------------------------------------------


질문 처리 중 (파인튜닝 전 모델):  99%|█████████▉| 129/130 [1:04:23<00:31, 31.24s/it]


📌 질문: 석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 알려주세요."
💡 답변: 석고수정이 발생하는 가장 큰 원인은 일반적으로 다음과 같습니다:

1. 과도한 석고 섭취: 석고 섭취가 너무 많으면 석고가 체내에 과도하게 축적되어 석고수정이 발생할 수 있습니다.
2. 석고 섭취와 석고 섭취 사이의 불일치: 석고 섭취가 석고 섭취보다 많을 때 석고 수정이 발생할 수 있습니다.
3. 석고 섭취와 석고 섭취 사이의 불일치: 석고 섭취가 석고 섭취보다 적을 때 석고 수정이 발생할 수 있습니다.
4. 석고 섭취와 석고 섭취 사이의 불일치: 석고 섭취가 석고 섭취보다 적을 때 석고 수정이 발생할 수 있습니다.
5. 석고 섭취와 석고 섭취 사이의 불일치: 석고 섭취가 석고 섭취보다 적을 때 석고 수정이 발생할 수 있습니다.

석고수정 문제를 해결하는 방법은 다음과 같습니다:

1. 석고 섭취를 줄이기: 석고 섭취를 줄이는 것이 석고 수정 문제를 해결하는 데 도움이 될 수 있습니다. 석고 섭취를 줄이는 방법은 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄이는 것뿐만 아니라 석고 섭취를 줄
--------------------------------------------------------------------------------


질문 처리 중 (파인튜닝 전 모델): 100%|██████████| 130/130 [1:04:38<00:00, 29.84s/it]


📌 질문: 카페트의 기대 수명은 얼마나 될까요? 그리고 오리지널징크의 장점에는 무엇이 있나요?
💡 답변: 카페트의 기대 수명은 일반적으로 약 20년입니다. 그러나 이는 카페트의 품질과 사용 방법에 크게 의존합니다. 카페트의 품질을 유지하기 위해 적절한 관리와 정기적인 청소가 필요합니다.

오리지널징크의 장점은 다음과 같습니다:

1. 유연성: 오리지널징크는 다양한 형태와 크기에 맞춰질 수 있으며, 이는 다양한 디자인 요구에 적합합니다.
2. 손감감: 오리지널징크는 일반적인 직물보다 더 손감감이므로, 더 부드럽고 편안한 느낌을 제공합니다.
3. 재활용 가능성: 오리지널징크는 재활용된 직물로 만들어지므로, 환경에 미치는 영향을 줄일 수 있습니다.
4. 지속 가능성: 오리지널징크는 재생 가능 자원으로 만들어지므로, 지속 가능한 옵션으로서의 장점이 있습니다.
5. 디자인의 다양성: 오리지널징크는 다양한 색상과 패턴으로 제공되므로, 창의적인 디자인을 위해 유용합니다.
--------------------------------------------------------------------------------
✅ submission_baseline.csv 저장 완료! (파인튜닝 전 모델 결과)


### LoRA 설정

In [ ]:
from peft import LoraConfig, get_peft_model
# from peft import get_peft_model :
# 파라미터를 다 학습하지 않고, 일부 작은 모듈만 학습해서 메모리 절약 + 속도 향상 + 성능 유지를 목표로 하는 라이브러리

lora_config =LoraConfig(
    r = 32,
    target_modules = ['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_alpha = 64,
    lora_dropout = 0.05, # lora_dropout=0.05 ~ 0.1 정도로 시작
    task_type = 'CAUSAL_LM'
)

### 모델 로드 및 LoRA 적용

In [ ]:
model = get_peft_model(model, lora_config) # 모델에 LoRA 어댑터 추가, 학습 준비 단계

In [ ]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments
# from transformers import DataCollatorForLanguageModeling : 배치 안의 모든 시퀀스 길이를 맞춰주는 역할
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=False : 앞에서부터 순차 예측

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=1, # GPU 1개당 배치 사이즈 1
    gradient_accumulation_steps=4, # 4 step 동안 gradient를 누적 후 1번 업데이트
    warmup_steps=10,
    max_steps=50, # 50 step 학습 후 종료
    learning_rate=3e-4, # AdamW 옵티마이저의 학습률
    fp16=True,
    logging_steps=1,
    output_dir='outputs',
    optim='paged_adamw_8bit',
    report_to = 'none'
)

In [ ]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=test,
    peft_config=lora_config,
    args=training_args,
    data_collator=data_collator
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'wjdgnl/Phi-4-mini-instruct-4bit' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/5152 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1288 [00:00<?, ? examples/s]

Step,Training Loss
1,3.121200
2,3.134500
3,3.081000
4,2.833800
5,3.001800
6,2.735700
7,2.453800
8,2.890400
9,2.383300
10,2.291500


TrainOutput(global_step=50, training_loss=2.2291317701339723, metrics={'train_runtime': 110.3783, 'train_samples_per_second': 1.812, 'train_steps_per_second': 0.453, 'total_flos': 478696504504320.0, 'train_loss': 2.2291317701339723})

### LoRA 결과 확인

In [ ]:
# 파인튜닝 결과 확인

def get_completion(query : str, model, tokenizer) :

  prompt_template = '''<|system|>Your are a helpful assistant.<|end|><|user|>{query}<|end|><|assistant|>''' # 모델에 따라 바뀜

  prompt = prompt_template.format(query = query)
  encodeds = tokenizer(prompt, return_tensors = 'pt', add_special_tokens = True)
  model_inputs = encodeds.to('cuda:0')
  generated_ids = model.generate(**model_inputs, max_new_tokens = 512)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens = True)
  return decoded

# fine-tuning 이후
result = get_completion(query= '방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?',
                        model=trainer.model, # # trainer.model 은 LoRA 어댑터가 적용된 모델
                        tokenizer=tokenizer)
print(result)

Your are a helpful assistant.방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?방청 페인트는 일반적으로 2단계 페인트로 분류되며, 1단계 페인트는 2단계 페인트보다 저렴하지만, 2단계 페인트는 1단계 페인트보다 더 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 


Your are a helpful assistant.방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?방청 페인트는 일반적으로 2단계 페인트로 분류되며, 1단계 페인트는 2단계 페인트보다 저렴하지만, 2단계 페인트는 1단계 페인트보다 더 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는 1단계 페인트보다 내구성이 뛰어나고, 2단계 페인트는

### LoRA + 기본모델 결과 확인

In [ ]:
merged_model = trainer.model.merge_and_unload()

# 파인튜닝 결과 확인

def get_completion(query : str, model, tokenizer) :

  prompt_template = '''<|system|>Your are a helpful assistant.<|end|><|user|>{query}<|end|><|assistant|>''' # 모델에 따라 바뀜

  prompt = prompt_template.format(query = query)
  encodeds = tokenizer(prompt, return_tensors = 'pt', add_special_tokens = True)
  model_inputs = encodeds.to('cuda:0')
  generated_ids = model.generate(**model_inputs, max_new_tokens = 512)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens = True)
  return decoded

# fine-tuning 이후
result = get_completion(query= '방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?',
                        model=merged_model, # # trainer.model 은 LoRA 어댑터가 적용된 모델
                        tokenizer=tokenizer)
print(result)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Your are a helpful assistant.방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?방청 페인트에는 여러 종류가 있으며, 선택은 프로젝트의 요구 사항과 개인의 선호에 따라 달라집니다. 일반적인 종류는 다음과 같습니다:

1. 유리 페인트: 유리 페인트는 유리 표면에 부착하여 유리처럼 보이는 효과를 제공하는 페인트입니다. 유리 페인트는 다양한 색상 옵션과 유리 표면에 부착하는 방식으로 유리처럼 보이는 효과를 제공하는 것으로 유명합니다.

2. 유리 페인트: 유리 페인트는 유리 표면에 부착하여 유리처럼 보이는 효과를 제공하는 페인트입니다. 유리 페인트는 다양한 색상 옵션과 유리 표면에 부착하는 방식으로 유리처럼 보이는 효과를 제공하는 것으로 유명합니다.

3. 유리 페인트: 유리 페인트는 유리 표면에 부착하여 유리처럼 보이는 효과를 제공하는 페인트입니다. 유리 페인트는 다양한 색상 옵션과 유리 표면에 부착하는 방식으로 유리처럼 보이는 효과를 제공하는 것으로 유명합니다.

4. 유리 페인트: 유리 페인트는 유리 표면에 부착하여 유리


## 데이콘 코드 참고

In [ ]:
from peft import LoraConfig, get_peft_model
lora_config =LoraConfig(
    lora_alpha=256,
    lora_dropout=0.05,
    r=128,
    target_modules=['v_proj', 'up_proj', 'down_proj', 'k_proj', 'o_proj', 'q_proj', 'gate_proj'],
    bias="none",
    task_type="CAUSAL_LM"
)


In [ ]:
model = get_peft_model(model, lora_config) # 모델에 LoRA 어댑터 추가, 학습 준비 단계

In [ ]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments
# from transformers import DataCollatorForLanguageModeling : 배치 안의 모든 시퀀스 길이를 맞춰주는 역할
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=False : 앞에서부터 순차 예측

In [ ]:
# 데이콘 코드(GPU 용량 부족으로 수정)

train_params = TrainingArguments(
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    max_steps=900, # 880이 최대인 듯
    learning_rate=3e-4,
    fp16=True,                           # mixed precision
    logging_steps=10,
    save_strategy='epoch',
    optim='paged_adamw_8bit',            # 8bit optimizer
    max_grad_norm=0.3,
    lr_scheduler_type='cosine',
    report_to='none',
    seed = 2025
)


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from trl import SFTTrainer
from transformers import DataCollatorForLanguageModeling, TrainingArguments

trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    peft_config=lora_config,
    args=train_params,
    data_collator=data_collator
)


trainer.train()

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'wjdgnl/Phi-4-mini-instruct-4bit' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/5152 [00:00<?, ? examples/s]

Step,Training Loss
10,2.447000
20,2.053600
30,1.979200
40,1.967200
50,1.895400
60,1.746700
70,1.725500
80,1.772400
90,1.751600
100,1.707700


TrainOutput(global_step=900, training_loss=1.342338923348321, metrics={'train_runtime': 1960.215, 'train_samples_per_second': 1.837, 'train_steps_per_second': 0.459, 'total_flos': 9238513214730240.0, 'train_loss': 1.342338923348321})

### LoRA 결과 확인

In [ ]:
# 파인튜닝 결과 확인

def get_completion(query : str, model, tokenizer) :

  prompt_template = '''<|system|>Your are a helpful assistant.<|end|><|user|>{query}<|end|><|assistant|>''' # 모델에 따라 바뀜

  prompt = prompt_template.format(query = query)
  encodeds = tokenizer(prompt, return_tensors = 'pt', add_special_tokens = True)
  model_inputs = encodeds.to('cuda:0')
  generated_ids = model.generate(**model_inputs, max_new_tokens = 512)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens = True)
  return decoded

# fine-tuning 이후
result = get_completion(query= '방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?',
                        model=trainer.model, # # trainer.model 은 LoRA 어댑터가 적용된 모델
                        tokenizer=tokenizer)
print(result)

Your are a helpful assistant.방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?방청 페인트의 종류로는 주로 수성방청페인트, 유성방청페인트, 전색제방청페인트가 사용됩니다. 수성방청페인트는 물을 기반으로 만들어진 페인트로, 환경에 더 친화적입니다. 유성방청페인트는 유기물을 기반으로 만들어져 있으며, 내구성이 뛰어나고 내수성이 강한 특징을 가지고 있습니다. 마지막으로 전색제방청페인트는 전색제를 사용하여 만들어지며, 다양한 색상과 표면의 강도를 제공합니다. 이러한 종류의 방청페인트를 선택할 때에는 사용 환경과 목적에 맞게 적합한 제품을 선택하는 것이 중요합니다. 또한, 방청페인트의 종류를 선택할 때에는 페인트의 내구성과 내수성, 그리고 사용 환경에 적합한 제품을 선택하는 것이 중요합니다. 원목사이딩은 자연스러운 느낌을 주고, 다양한 색상과 질감을 제공하는 장점이 있지만, 습기에 취약하고 유지보수가 어렵다는 단점이 있습니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려

Your are a helpful assistant.방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?방청 페인트의 종류로는 주로 수성방청페인트, 유성방청페인트, 전색제방청페인트가 사용됩니다. 수성방청페인트는 물을 기반으로 만들어진 페인트로, 환경에 더 친화적입니다. 유성방청페인트는 유기물을 기반으로 만들어져 있으며, 내구성이 뛰어나고 내수성이 강한 특징을 가지고 있습니다. 마지막으로 전색제방청페인트는 전색제를 사용하여 만들어지며, 다양한 색상과 표면의 강도를 제공합니다. 이러한 종류의 방청페인트를 선택할 때에는 사용 환경과 목적에 맞게 적합한 제품을 선택하는 것이 중요합니다. 또한, 방청페인트의 종류를 선택할 때에는 페인트의 내구성과 내수성, 그리고 사용 환경에 적합한 제품을 선택하는 것이 중요합니다. 원목사이딩은 자연스러운 느낌을 주고, 다양한 색상과 질감을 제공하는 장점이 있지만, 습기에 취약하고 유지보수가 어렵다는 단점이 있습니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩을 사용할 경우에는 적절한 관리와 보수가 필요하며, 습기에 취약하다는 점을 고려해야 합니다. 따라서, 원목사이딩

### 성능 평가

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# 1. 테스트 데이터 불러오기
test_df = pd.read_csv("test.csv")   # 반드시 'id', '질문' 컬럼 포함

# 2. 답변 생성 함수 (파인튜닝된 모델 활용)
def get_completion(query: str, model, tokenizer):
    # 프롬프트에는 system 역할 포함
    prompt_template = "<|system|>You are a helpful assistant.<|end|><|user|>{query}<|end|><|assistant|>"
    prompt = prompt_template.format(query=query)

    encodeds = tokenizer(prompt, return_tensors='pt', add_special_tokens=True)
    model_inputs = encodeds.to("cuda:0")

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,   # 답변 길이 적절히 제한
            do_sample=False
        )

    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # system 메시지 제거
    decoded = decoded.replace("You are a helpful assistant.", "").strip()
    # 질문 반복 제거
    if query in decoded:
        decoded = decoded.split(query)[-1].strip()
    # 혹시 <|assistant|> 토큰이 남아있으면 제거
    if "<|assistant|>" in decoded:
        decoded = decoded.split("<|assistant|>")[-1].strip()

    return decoded

# 3. 임베딩 모델 로드 (512차원 출력)
embedding_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

# 4. 테스트 데이터 → 답변 생성 → 임베딩 변환
embeddings_list = []
answers_list = []   # 답변 텍스트 저장용

for q in tqdm(test_df["질문"], desc="질문 처리 중"):
    answer = get_completion(q, trainer.model, tokenizer)      # 답변 생성
    vec = embedding_model.encode(answer)                      # shape (512,)
    embeddings_list.append(vec)
    answers_list.append(answer)

    # 진행 상황에 질문 + 답변 함께 출력
    print("\n📌 질문:", q)
    print("💡 답변:", answer)
    print("-" * 80)

# 5. 임베딩을 DataFrame으로 변환 (dim_0 ~ dim_511)
embedding_df = pd.DataFrame(embeddings_list, columns=[f"dim_{i}" for i in range(512)])

# 6. 제출용 DataFrame (id + 512차원)
submission = pd.concat([test_df[["id"]], embedding_df], axis=1)

# 7. CSV 저장
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv 저장 완료! (형식: id + dim_0 ~ dim_511)")

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...


📌 질문: 평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어떤 장점이 있나요?
💡 답변: 수성 벽체용 탄성 방수 도료는 물의 침투를 막아주어 누수가 발생하지 않도록 도료를 바르는 것이 가장 효과적입니다. 또한, 탄성 방수 도료는 물이 빠져나가거나 물이 빠져나가지 않도록 도료를 바르는 것이 중요합니다. 이를 통해 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 또한, 탄성 방수 도료는 물의 침투를 막아주어 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 이를 통해 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 또한, 탄성 방수 도료는 물의 침투를 막아주어 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 이를 통해 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 또한, 탄성 방수 도료는 물의 침투를 막아주어 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 이를 통해 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 또한, 탄성 방수 도료는 물의 침투를 막아주어 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 이를 통해 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 또한, 탄성 방수 도료는 물의 침투를 막아주어 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 이를 통해 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 또한, 탄성 방수 도료는 물의 침투를 막아주어 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 이를 통해 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 또한, 탄성 방수 도료는 물의 침투를 막아주어 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 이를 통해 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 또한, 탄성 방수 도료는 물의 침투를 막아주어 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 이를 통해 누수가 발생하지 않도록 도료를 바르는 것이 중요합니다. 또한, 탄성 방수 도료는 물의 침투를 막

질문 처리 중:  99%|█████████▉| 129/130 [1:28:28<00:41, 41.05s/it]


📌 질문: 석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 알려주세요."
💡 답변: 석고수정이 발생하는 가장 큰 원인은 석고의 내구성에 있습니다. 석고는 습기에 취약하고, 특히 석고수정이 발생하는 환경은 습도가 높거나, 석고가 물에 노출되면 변형될 수 있습니다. 이러한 석고수정이 발생하는 것을 방지하기 위해서는 건조한 환경을 유지하고, 석고를 적절히 보관하는 것이 중요합니다. 또한, 석고를 사용할 때에는 물을 주지 않도록 주의해야 합니다. 만약 석고가 물에 노출되면 변형될 수 있으므로, 물을 주지 않도록 주의해야 합니다. 이러한 석고수정이 발생하는 것을 방지하기 위해서는 건조한 환경을 유지하고, 석고를 적절히 보관하는 것이 중요합니다. 또한, 석고를 사용할 때에는 물을 주지 않도록 주의해야 합니다. 만약 석고가 물에 노출되면 변형될 수 있으므로, 물을 주지 않도록 주의해야 합니다. 이러한 석고수정이 발생하는 것을 방지하기 위해서는 건조한 환경을 유지하고, 석고를 적절히 보관하는 것이 중요합니다. 또한, 석고를 사용할 때에는 물을 주지 않도록 주의해야 합니다. 만약 석고가 물에 노출되면 변형될 수 있으므로, 물을 주지 않도록 주의해야 합니다. 이러한 석고수정이 발생하는 것을 방지하기 위해서는 건조한 환경을 유지하고, 석고를 적절히 보관하는 것이 중요합니다. 또한, 석고를 사용할 때에는 물을 주지 않도록 주의해야 합니다. 만약 석고가 물에 노출되면 변형될 수 있으므로, 물을 주지 않도록 주의해야 합니다. 이러한 석고수정이 발생하는 것을 방지하기 위해서는 건조한 환경을 유지하고, 석고를 적절히 보관하는 것이 중요합니다. 또한, 석고를 사용할 때에는 물을 주지 않도록 주의해야 합니다. 만약 석고가 물에 노출되면 변형될 수 있으므로, 물을 주지 않도록 주의해야 합니다. 이러한 석고수정이 발생하는 것을 방지
------------------------------------------------------------------------

질문 처리 중: 100%|██████████| 130/130 [1:28:46<00:00, 40.97s/it]


📌 질문: 카페트의 기대 수명은 얼마나 될까요? 그리고 오리지널징크의 장점에는 무엇이 있나요?
💡 답변: 카페트의 기대수명은 일반적으로 5년입니다. 그러나 사용 환경과 유지보수 여부에 따라 다를 수 있습니다. 오리지널징크는 카페트와 비슷한 디자인을 가지고 있지만, 오리지널징크는 가벼우면서도 강도가 뛰어나고 내구성이 뛰어나며, 다양한 색상과 디자인이 가능하다는 장점이 있습니다. 또한 오리지널징크는 친환경적이며, 자석에 자주 사용되는 소재로서 친환경적인 면에서도 장점이 있습니다. 이러한 이유로 오리지널징크는 카페트와 비교하여 더 오랫동안 사용할 수 있는 장점이 있습니다. 그러나 오리지널징크는 가격이 상대적으로 높을 수 있으며, 자석에 자주 사용되는 소재로서 자석에 부착되기 쉽지 않을 수 있다는 단점도 있습니다. 따라서 사용 환경과 유지보수 여부에 따라 적합한 소재를 선택하는 것이 중요합니다.
--------------------------------------------------------------------------------
✅ submission.csv 저장 완료! (형식: id + dim_0 ~ dim_511)


### LoRA + 기본모델 결과 확인

In [ ]:
merged_model = trainer.model.merge_and_unload()

# 파인튜닝 결과 확인

def get_completion(query : str, model, tokenizer) :

  prompt_template = '''<|system|>Your are a helpful assistant.<|end|><|user|>{query}<|end|><|assistant|>''' # 모델에 따라 바뀜

  prompt = prompt_template.format(query = query)
  encodeds = tokenizer(prompt, return_tensors = 'pt', add_special_tokens = True)
  model_inputs = encodeds.to('cuda:0')
  generated_ids = model.generate(**model_inputs, max_new_tokens = 512)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens = True)
  return decoded

# fine-tuning 이후
result = get_completion(query= '방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?',
                        model=merged_model, # # trainer.model 은 LoRA 어댑터가 적용된 모델
                        tokenizer=tokenizer)
print(result)

### 2차

In [ ]:
from peft import LoraConfig, get_peft_model
# from peft import get_peft_model :
# 파라미터를 다 학습하지 않고, 일부 작은 모듈만 학습해서 메모리 절약 + 속도 향상 + 성능 유지를 목표로 하는 라이브러리

lora_config = LoraConfig(
    r=32,
    target_modules=['o_proj','q_proj','v_proj','k_proj','up_proj','down_proj','gate_proj'],
    lora_alpha=64,
    lora_dropout=0.05,
    task_type='CAUSAL_LM'
)

In [ ]:
model = get_peft_model(model, lora_config) # 모델에 LoRA 어댑터 추가, 학습 준비 단계

from transformers import DataCollatorForLanguageModeling, TrainingArguments
# from transformers import DataCollatorForLanguageModeling : 배치 안의 모든 시퀀스 길이를 맞춰주는 역할
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=False : 앞에서부터 순차 예측

training_args = TrainingArguments(
    per_device_train_batch_size=1, # GPU 1개당 배치 사이즈 1
    gradient_accumulation_steps=8, # 4 step 동안 gradient를 누적 후 1번 업데이트
    warmup_steps=50,
    max_steps=500, # 50 step 학습 후 종료
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    output_dir='outputs',
    optim='paged_adamw_8bit',
    report_to = 'none'
)

In [ ]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=test,
    peft_config=lora_config,
    args=training_args,
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'wjdgnl/Phi-4-mini-instruct-4bit' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/5152 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1288 [00:00<?, ? examples/s]

Step,Training Loss
10,2.991500
20,2.641700
30,2.252300
40,1.972400
50,2.012200
60,1.890300
70,1.816800
80,1.853300
90,1.778800
100,1.710600


TrainOutput(global_step=500, training_loss=1.6106539363861083, metrics={'train_runtime': 2052.4769, 'train_samples_per_second': 1.949, 'train_steps_per_second': 0.244, 'total_flos': 9970336663609344.0, 'train_loss': 1.6106539363861083})

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# 1. 테스트 데이터 불러오기
test_df = pd.read_csv("test.csv")   # 반드시 'id', '질문' 컬럼 포함

# 2. 답변 생성 함수 (파인튜닝된 모델 활용)
def get_completion(query: str, model, tokenizer):
    # 프롬프트에는 system 역할 포함
    prompt_template = "<|system|>You are a helpful assistant.<|end|><|user|>{query}<|end|><|assistant|>"
    prompt = prompt_template.format(query=query)

    encodeds = tokenizer(prompt, return_tensors='pt', add_special_tokens=True)
    model_inputs = encodeds.to("cuda:0")

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,   # 답변 길이 적절히 제한
            do_sample=False
        )

    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # system 메시지 제거
    decoded = decoded.replace("You are a helpful assistant.", "").strip()
    # 질문 반복 제거
    if query in decoded:
        decoded = decoded.split(query)[-1].strip()
    # 혹시 <|assistant|> 토큰이 남아있으면 제거
    if "<|assistant|>" in decoded:
        decoded = decoded.split("<|assistant|>")[-1].strip()

    return decoded

# 3. 임베딩 모델 로드 (512차원 출력)
embedding_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

# 4. 테스트 데이터 → 답변 생성 → 임베딩 변환
embeddings_list = []
answers_list = []   # 답변 텍스트 저장용

for q in tqdm(test_df["질문"], desc="질문 처리 중"):
    answer = get_completion(q, trainer.model, tokenizer)      # 답변 생성
    vec = embedding_model.encode(answer)                      # shape (512,)
    embeddings_list.append(vec)
    answers_list.append(answer)

    # 진행 상황에 질문 + 답변 함께 출력
    print("\n📌 질문:", q)
    print("💡 답변:", answer)
    print("-" * 80)

# 5. 임베딩을 DataFrame으로 변환 (dim_0 ~ dim_511)
embedding_df = pd.DataFrame(embeddings_list, columns=[f"dim_{i}" for i in range(512)])

# 6. 제출용 DataFrame (id + 512차원)
submission = pd.concat([test_df[["id"]], embedding_df], axis=1)

# 7. CSV 저장
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv 저장 완료! (형식: id + dim_0 ~ dim_511)")

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

질문 처리 중:   0%|          | 0/130 [00:36<?, ?it/s]


KeyboardInterrupt: 

### 3차

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    target_modules=['o_proj','q_proj','v_proj','k_proj','up_proj','down_proj','gate_proj'],
    lora_alpha=128,
    lora_dropout=0.05,
    task_type='CAUSAL_LM'
)

In [ ]:
model = get_peft_model(model, lora_config) # 모델에 LoRA 어댑터 추가, 학습 준비 단계

from transformers import DataCollatorForLanguageModeling, TrainingArguments
# from transformers import DataCollatorForLanguageModeling : 배치 안의 모든 시퀀스 길이를 맞춰주는 역할
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=False : 앞에서부터 순차 예측

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=50,
    max_steps=800,
    learning_rate=1.5e-4,
    fp16=True,
    logging_steps=10,
    output_dir='outputs',
    optim='paged_adamw_8bit',
    report_to='none'
)

In [ ]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=test,
    peft_config=lora_config,
    args=training_args,
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'wjdgnl/Phi-4-mini-instruct-4bit' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/5152 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1288 [00:00<?, ? examples/s]

Step,Training Loss
10,2.963900
20,2.601800
30,2.233500
40,2.167200
50,2.027400
60,1.950800
70,1.849600
80,1.782500
90,1.947800
100,1.846600


OutOfMemoryError: CUDA out of memory. Tried to allocate 360.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 2.12 MiB is free. Process 3679 has 14.62 GiB memory in use. Of the allocated memory 14.16 GiB is allocated by PyTorch, and 331.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### 파인튜닝 적용 전 성능

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=32,
    target_modules=['o_proj','q_proj','v_proj','k_proj','up_proj','down_proj','gate_proj'],
    lora_alpha=64,
    task_type='CAUSAL_LM',
    bias="none"
)

model = get_peft_model(model, lora_config) # 모델에 LoRA 어댑터 추가, 학습 준비 단계

from transformers import DataCollatorForLanguageModeling, TrainingArguments
# from transformers import DataCollatorForLanguageModeling : 배치 안의 모든 시퀀스 길이를 맞춰주는 역할
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=False : 앞에서부터 순차 예측

training_args = TrainingArguments(
    per_device_train_batch_size=1,   # T4 GPU 안전
    gradient_accumulation_steps=8,   # effective batch size 8
    max_steps=500,                    # 1차와 동일, 충분한 학습
    warmup_steps=50,                  # 학습 초반 안정화
    learning_rate=3e-4,               # 1차와 동일, 빠른 수렴 가능
    fp16=True,
    logging_steps=10,
    save_strategy='steps',
    save_steps=300,                   # 중간 체크포인트
    output_dir='outputs',
    optim='paged_adamw_8bit',         # 메모리 절약
    lr_scheduler_type='cosine',
    max_grad_norm=0.3,
    report_to='none',
    seed=2025
)

from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=test,
    peft_config=lora_config,
    args=training_args,
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'wjdgnl/Phi-4-mini-instruct-4bit' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/5152 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1288 [00:00<?, ? examples/s]

Step,Training Loss
10,2.752700
20,2.487800
30,2.094000
40,1.923700
50,1.906300
60,1.920800
70,1.879800
80,1.782500
90,1.690000
100,1.674200


OutOfMemoryError: CUDA out of memory. Tried to allocate 568.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 260.12 MiB is free. Process 9218 has 14.36 GiB memory in use. Of the allocated memory 14.07 GiB is allocated by PyTorch, and 161.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# 1. 테스트 데이터 불러오기
test_df = pd.read_csv("test.csv")   # 반드시 'id', '질문' 컬럼 포함

# 2. 답변 생성 함수 (파인튜닝된 모델 활용)
def get_completion(query: str, model, tokenizer):
    # 프롬프트에는 system 역할 포함
    prompt_template = "<|system|>You are a helpful assistant.<|end|><|user|>{query}<|end|><|assistant|>"
    prompt = prompt_template.format(query=query)

    encodeds = tokenizer(prompt, return_tensors='pt', add_special_tokens=True)
    model_inputs = encodeds.to("cuda:0")

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,   # 답변 길이 적절히 제한
            do_sample=False
        )

    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # system 메시지 제거
    decoded = decoded.replace("You are a helpful assistant.", "").strip()
    # 질문 반복 제거
    if query in decoded:
        decoded = decoded.split(query)[-1].strip()
    # 혹시 <|assistant|> 토큰이 남아있으면 제거
    if "<|assistant|>" in decoded:
        decoded = decoded.split("<|assistant|>")[-1].strip()

    return decoded

# 3. 임베딩 모델 로드 (512차원 출력)
embedding_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

# 4. 테스트 데이터 → 답변 생성 → 임베딩 변환
embeddings_list = []
answers_list = []   # 답변 텍스트 저장용

for q in tqdm(test_df["질문"], desc="질문 처리 중"):
    answer = get_completion(q, trainer.model, tokenizer)      # 답변 생성
    vec = embedding_model.encode(answer)                      # shape (512,)
    embeddings_list.append(vec)
    answers_list.append(answer)

    # 진행 상황에 질문 + 답변 함께 출력
    print("\n📌 질문:", q)
    print("💡 답변:", answer)
    print("-" * 80)

# 5. 임베딩을 DataFrame으로 변환 (vec_0 ~ vec_511)
embedding_df = pd.DataFrame(embeddings_list, columns=[f"vec_{i}" for i in range(512)])

# 6. 제출용 DataFrame (id + 512차원)
submission = pd.concat([test_df[["id"]], embedding_df], axis=1)

# 7. CSV 저장
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv 저장 완료! (형식: id + vec_0 ~ vec_511)")

### 4차

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=48,
    target_modules=['o_proj','q_proj','v_proj','k_proj','up_proj','down_proj','gate_proj'],
    lora_alpha=96,
    lora_dropout=0.05,
    task_type='CAUSAL_LM',
    bias="none"
)

model = get_peft_model(model, lora_config) # 모델에 LoRA 어댑터 추가, 학습 준비 단계

from transformers import DataCollatorForLanguageModeling, TrainingArguments
# from transformers import DataCollatorForLanguageModeling : 배치 안의 모든 시퀀스 길이를 맞춰주는 역할
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=False : 앞에서부터 순차 예측

training_args = TrainingArguments(
    per_device_train_batch_size=1,   # T4 GPU 안전
    gradient_accumulation_steps=8,   # effective batch size 8
    max_steps=900,                    # 1차와 동일, 충분한 학습
    warmup_steps=50,                  # 학습 초반 안정화
    learning_rate=3e-4,               # 1차와 동일, 빠른 수렴 가능
    fp16=True,
    logging_steps=10,
    save_strategy='steps',
    save_steps=300,                   # 중간 체크포인트
    output_dir='outputs',
    optim='paged_adamw_8bit',         # 메모리 절약
    lr_scheduler_type='cosine',
    max_grad_norm=0.3,
    report_to='none',
    seed=2025
)

from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=test,
    peft_config=lora_config,
    args=training_args,
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'wjdgnl/Phi-4-mini-instruct-4bit' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/5152 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1288 [00:00<?, ? examples/s]

Step,Training Loss
10,2.721600
20,2.392800
30,2.026100
40,1.898600
50,1.888400
60,1.892200
70,1.857400
80,1.752300
90,1.665200
100,1.648000


TrainOutput(global_step=900, training_loss=1.198847065501743, metrics={'train_runtime': 3730.5487, 'train_samples_per_second': 1.93, 'train_steps_per_second': 0.241, 'total_flos': 1.813845498227712e+16, 'train_loss': 1.198847065501743})

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# 1. 테스트 데이터 불러오기
test_df = pd.read_csv("test.csv")   # 반드시 'id', '질문' 컬럼 포함

# 2. 답변 생성 함수 (파인튜닝된 모델 활용)
def get_completion(query: str, model, tokenizer):
    # 프롬프트에는 system 역할 포함
    prompt_template = "<|system|>You are a helpful assistant.<|end|><|user|>{query}<|end|><|assistant|>"
    prompt = prompt_template.format(query=query)

    encodeds = tokenizer(prompt, return_tensors='pt', add_special_tokens=True)
    model_inputs = encodeds.to("cuda:0")

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,   # 답변 길이 적절히 제한
            do_sample=False
        )

    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # system 메시지 제거
    decoded = decoded.replace("You are a helpful assistant.", "").strip()
    # 질문 반복 제거
    if query in decoded:
        decoded = decoded.split(query)[-1].strip()
    # 혹시 <|assistant|> 토큰이 남아있으면 제거
    if "<|assistant|>" in decoded:
        decoded = decoded.split("<|assistant|>")[-1].strip()

    return decoded

# 3. 임베딩 모델 로드 (512차원 출력)
embedding_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

# 4. 테스트 데이터 → 답변 생성 → 임베딩 변환
embeddings_list = []
answers_list = []   # 답변 텍스트 저장용

for q in tqdm(test_df["질문"], desc="질문 처리 중"):
    answer = get_completion(q, trainer.model, tokenizer)      # 답변 생성
    vec = embedding_model.encode(answer)                      # shape (512,)
    embeddings_list.append(vec)
    answers_list.append(answer)

    # 진행 상황에 질문 + 답변 함께 출력
    print("\n📌 질문:", q)
    print("💡 답변:", answer)
    print("-" * 80)

# 5. 임베딩을 DataFrame으로 변환 (vec_0 ~ vec_511)
embedding_df = pd.DataFrame(embeddings_list, columns=[f"vec_{i}" for i in range(512)])

# 6. 제출용 DataFrame (id + 512차원)
submission = pd.concat([test_df[["id"]], embedding_df], axis=1)

# 7. CSV 저장
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv 저장 완료! (형식: id + vec_0 ~ vec_511)")

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...


📌 질문: 평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어떤 장점이 있나요?
💡 답변: 탄성 방수 도료를 사용하면 도막이 늘어나거나 수축되는 탄성 있는 특성을 가지고 있어, 벽면에 균일하게 도포하여 누수가 발생하는 문제를 방지할 수 있습니다. 또한, 탄성 방수 도료는 물의 침투를 막아주고, 벽면을 보호하여 수분으로 인한 손상을 방지하는 데 도움이 됩니다. 따라서, 탄성 방수 도료를 사용하면 벽면의 누수 문제를 효과적으로 해결할 수 있습니다.  또한, 탄성 방수 도료는 물에 대한 내구성을 높여주어 장기간의 보호를 제공하여 건물의 수명을 연장시킬 수 있는 장점이 있습니다.  따라서, 탄성 방수 도료를 사용하여 벽면의 누수 문제를 방지하고, 건물의 수명을 연장시킬 수 있는 장점이 있습니다.  따라서, 탄성 방수 도료를 사용하여 벽면의 누수 문제를 방지하고, 건물의 수명을 연장시킬 수 있는 장점이 있습니다.  따라서, 탄성 방수 도료를 사용하여 벽면의 누수 문제를 방지하고, 건물의 수명을 연장시킬 수 있는 장점이 있습니다.  따라서, 탄성 방수 도료를 사용하여 벽면의 누수 문제를 방지하고, 건물의 수명을 연장시킬 수 있는 장점이 있습니다.  따라서, 탄성 방수 도료를 사용하여 벽면의 누수 문제를 방지하고, 건물의 수명을 연장시킬 수 있는 장점이 있습니다.  따라서, 탄성 방수 도료를 사용하여 벽면의 누수 문제를 방지하고, 건물의 수명을 연장시킬 수 있는 장점이 있습니다.  따라서, 탄성 방수 도료를 사용하여 벽면의 누수 문제를 방지하고, 건물의 수명을 연장시킬 수 있는 장점이 있습니다.  따라서, 탄성 방수 도료를 사용하여 벽면의 누수 문제를 방지하고, 건물의 수명을 연장시킬 수 있는 장점이 있습니다.  따라서, 탄성 방수 도료를 사용하여 벽면의 누수 문제를 방지하고, 건물의 수명을 연장시킬 수 있는
-------------------------------------------------------------------------

질문 처리 중:  99%|█████████▉| 129/130 [1:20:38<00:35, 35.73s/it]


📌 질문: 석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 알려주세요."
💡 답변: 석고수정이 발생하는 가장 큰 원인은 습도입니다. 석고수정이 발생하는 주된 이유는 석고의 물에 대한 민감한 특성 때문입니다. 석고는 물에 녹는 특성을 가지고 있어서 습기가 높으면 석고가 변형될 수 있습니다. 따라서, 습도 관리가 중요하며, 실내 습도를 적정 수준으로 유지하는 것이 석고수정 방지에 도움이 됩니다. 또한, 건조한 환경에서 보관 및 사용하는 것이 중요합니다. 만약 석고수정이 발생한다면 전문가의 도움을 받아 보수 작업을 하는 것이 좋습니다. 석고수정이 발생하는 경우에는 건조한 공간에서 보관하고, 습기 관리에 신경 써야 합니다. 만약 이러한 조치에도 불구하고 석고수정이 계속된다면 전문가의 도움을 받아 보수 작업을 하는 것이 좋습니다. 석고수정이 발생하는 경우에는 건조한 공간에서 보관하고, 습기 관리에 신경 써야 합니다. 만약 이러한 조치에도 불구하고 석고수정이 계속된다면 전문가의 도움을 받아 보수 작업을 하는 것이 좋습니다. 석고수정이 발생하는 경우에는 건조한 공간에서 보관하고, 습기 관리에 신경 써야 합니다. 만약 이러한 조치에도 불구하고 석고수정이 계속된다면 전문가의 도움을 받아 보수 작업을 하는 것이 좋습니다. 석고수정이 발생하는 경우에는 건조한 공간에서 보관하고, 습기 관리에 신경 써야 합니다. 만약 이러한 조치에도 불구하고 석고수정이 계속된다면 전문가의 도움을 받아 보수 작업을 하는 것이 좋습니다. 석고수정이 발생하는 경우에는 건조한 공간에서 보관하고, 습기 관리에 신경 써야 합니다. 만약 이러한 조치에도 불구하고 석고수정이 계속된다면 전문가의 도움을 받아 보수 작업을 하는 것이 좋습니다. 석고수정이 발생하는 경우에는 건조한 공간에서 보관하고, 습기 관리에 신경 써야 합니다. 만약 이러한 조치에도 불구하고 석고수정이 계속된다면 전문가의 도움
-------------------------------------------------

질문 처리 중: 100%|██████████| 130/130 [1:21:18<00:00, 37.53s/it]


📌 질문: 카페트의 기대 수명은 얼마나 될까요? 그리고 오리지널징크의 장점에는 무엇이 있나요?
💡 답변: 카페트의 기대수명은 일반적으로 6년입니다. 하지만 사용 환경, 관리 상태, 그리고 카페트의 품질에 따라 달라질 수 있습니다. 카페트를 정기적으로 청소하고 관리하는 것이 수명을 연장시키는 데 도움이 됩니다. 또한, 카페트의 종류와 소재에 따라 수명이 다를 수 있으므로, 사용 환경과 카페트의 특성을 고려하여 정기적인 유지보수를 하는 것이 중요합니다. 만약 카페트를 오랜 기간 사용 중에 변색이나 손상이 발생한다면, 전문가의 도움을 받아 보수작업을 하는 것이 좋습니다. 또한, 카페트의 소재와 사용 용도에 따라 수명을 연장하기 위한 추가적인 방법이 있을 수 있습니다. 예를 들어, 카페트를 가볍게 걸어 놓거나, 수분과 습기로부터 보호하는 등의 방법이 있습니다. 따라서, 카페트를 사용 중에는 주기적인 관리와 유지보수를 통해 수명을 연장시킬 수 있습니다. 만약 카페트를 오랜 기간 사용 중에 손상이 발생한다면, 전문가의 도움을 받아 보수작업을 하는 것이 좋습니다. 또한, 카페트의 소재와 사용 용도에 따라 수명을 연장하기 위한 추가적인 방법이 있을 수 있습니다. 예를 들어, 카페트를 가볍게 걸어 놓거나, 수분과 습기로부터 보호하는 등의 방법이 있습니다. 따라서, 카페트를 사용 중에는 주기적인 관리와 유지보수를 통해 수명을 연장시킬 수 있습니다. 만약 카페트를 오랜 기간 사용 중에 손상이 발생한다면, 전문가의 도움을 받아 보수작업을 하는 것이 좋습니다. 또한, 카페트의 소재와 사용 용도에 따라 수명을 연장하기 위한 추가적인 방법이 있을 수 있습니다. 예를 들어, 카페트를 가볍게 걸어 놓거나, 수분과 습기로부터 보호하는 등의 방법이 있습니다. 따라서, 카페트를 사용 중에는 주기적인 관리와 유지보수를 통해 수명을 연장시킬 수 있습니다. 만약 카페트를 오랜 기간 사용 중에 손상이 발생한다면, 전문가의 도움을 받아 보수작업을 하는 것이 좋습니다. 또한, 카페트의 소재와 사용 용도에